<a href="https://colab.research.google.com/github/apschlissel/w266-final-project/blob/main/T5_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T5 Classification - Reddit Data

Contains:
* T5 model creation
* Training on different n-sizes

In [3]:
!pip install -q transformers

     |████████████████████████████████| 3.8 MB 36.3 MB/s 
     |████████████████████████████████| 6.5 MB 54.3 MB/s 
     |████████████████████████████████| 596 kB 65.2 MB/s 
     |████████████████████████████████| 895 kB 61.7 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 


In [4]:
!pip install simpletransformers

     |████████████████████████████████| 249 kB 15.0 MB/s 
     |████████████████████████████████| 10.1 MB 53.4 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 325 kB 43.0 MB/s 
     |████████████████████████████████| 1.7 MB 49.2 MB/s 
     |████████████████████████████████| 1.2 MB 30.2 MB/s 
     |████████████████████████████████| 181 kB 12.1 MB/s 
     |████████████████████████████████| 144 kB 58.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
     |████████████████████████████████| 212 kB 44.6 MB/s 
     |████████████████████████████████| 136 kB 26.3 MB/s 
     |████████████████████████████████| 127 kB 58.8 MB/s 
     |████████████████████████████████| 271 kB 33.0 MB/s 
     |████████████████████████████████| 144 kB 51.2 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 164 kB 58.7 MB/s 
     |█████████████

In [5]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline
from simpletransformers.t5 import T5Model, T5Args
import pandas as pd
import logging
import numpy as np
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import json
import re
import random
import math
from bs4 import BeautifulSoup
# Pull reddit data from reddit api
import requests
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_rows', 100)

## Pull Reddit Data

In [6]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('pigKA_TKnDkXcatEGcbo8g', 'nawGKK2MfPtC6vKz8TjaNEnmYfAggA')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'Katsuuu100',
        'password': 'Testing159753'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [7]:
# Source: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c
# Source: https://pynative.com/python-random-choice/
# Pull from 5 classes. 5 classes = 5 subreddits.

my_list_of_dictionaries = []
total = 0
# Target Count for test set = 100.
n = int(math.ceil(100/0.20/25))

# url_list_check = [f"https://oauth.reddit.com/r/{five_random_subreddits[0]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[1]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[2]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[3]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[4]}/new/"
#           ]

url_list_check = [f"https://oauth.reddit.com/r/wallstreetbets/new/",
          f"https://oauth.reddit.com/r/teenagers/new/",
          f"https://oauth.reddit.com/r/copypasta/new/",
          f"https://oauth.reddit.com/r/genz/new/",
          f"https://oauth.reddit.com/r/unpopularopinion/new/",
          # f"https://oauth.reddit.com/r/frat/new/"
          ]

for i in range(len(url_list_check)):
    
  # print(url_list_check[i])
  res_check = requests.get(url_list_check[i],
                    headers=headers,
                    params={"limit": "1"})
  
  # print(res_check)
  # print(json.dumps(res_check.json()["data"]["children"][0]["data"]["name"], indent=4))
  name = res_check.json()["data"]["children"][0]["data"]["name"]
  page_count = 25
  
  for j in range(n):

    # url_list = [f"https://oauth.reddit.com/r/{five_random_subreddits[0]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[1]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[2]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[3]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[4]}/new/?count={page_count}&after={name}"
    #         ]

    url_list = [f"https://oauth.reddit.com/r/wallstreetbets/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/teenagers/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/copypasta/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/genz/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/unpopularopinion/new/?count={page_count}&after={name}",
            # f"https://oauth.reddit.com/r/frat/new/?count={page_count}&after={name}"
            ]
    
    print("Page Count:", page_count)
    print("Name:", name)
    print("Url:", url_list[i])
    
    res = requests.get(url_list[i],
                    headers=headers)
                    # params={"limit": "100"})

    reddit_dictionary = res.json()

    for k in range(len(reddit_dictionary["data"]["children"])):
      my_dictionary = {}
      my_dictionary["subreddit"] = reddit_dictionary["data"]["children"][k]["data"]["subreddit"]
      my_dictionary["text"] = reddit_dictionary["data"]["children"][k]["data"]["selftext"]
      # If a reddit post is has no body text:
      if my_dictionary["text"] == "":
        # Replace with title of reddit post.
        my_dictionary["text"] = reddit_dictionary["data"]["children"][k]["data"]["title"]
      
      print(my_dictionary["text"])
      # my_dictionary["title"] = reddit_dictionary["data"]["children"][k]["data"]["title"]
      # my_dictionary["url"] = reddit_dictionary["data"]["children"][k]["data"]["url"]
      # print(reddit_dictionary["data"]["children"][k]["data"]["subreddit"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["selftext"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["url"])
      my_list_of_dictionaries.append(my_dictionary)
      total += 1
      name = reddit_dictionary["data"]["children"][k]["data"]["name"]
    # print(json.dumps(my_list_of_dictionaries, indent=4, sort_keys=False))
    
    page_count += 25

print("Total gathered:", total)

Streaming output truncated to the last 5000 lines.
But you can tell the crowd just does not give a shit. There's not the excitement you're expecting, no real cheers, just sort of listening. They let him get through two songs before these 20-something Germans start complaining.

The crowd just starts chanting "Ve vant Veezer! Ve vant Veezer!" and Iggy Pop goes "Do you know who I fuckin am?" But they don't stop chanting. And he just stops mid-set because come on, why bother when you were supposed to be this exciting first act and the crowd just doesn't give a shit?

Anyway, "Veezer" comes out and they lose their shit. Germans, man.
 True story.

About  10-15 years ago, I was in Germany, at a Weezer concert. There was going  to be a surprise opening act, but it wasn't announced who it was gonna  be. So I get there and motherfuckin Iggy Pop comes onstage and starts  off with Lust for Life. I'm loving it.

But  you can tell the crowd just does not give a shit. There's not the  excitement yo

In [8]:
# Source: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb
# Convert list of dictionaries into pandas df
df = pd.DataFrame(my_list_of_dictionaries)
df.head()

,subreddit,text
0,wallstreetbets,Any good traded for a fellow ape
1,wallstreetbets,The top is in boys.
2,wallstreetbets,HMHC May 22.5C 💎🤲💎
3,wallstreetbets,This morning I bought $8K of $40 TWTR puts expiring end of month. Down 30% so far. let's see what happens.
4,wallstreetbets,Loading… Fresh New Sanctions


In [9]:
df['subreddit'].value_counts()

wallstreetbets      500
teenagers           500
copypasta           500
GenZ                500
unpopularopinion    500
Name: subreddit, dtype: int64

In [10]:
possible_labels = df.subreddit.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'GenZ': 3,
 'copypasta': 2,
 'teenagers': 1,
 'unpopularopinion': 4,
 'wallstreetbets': 0}

In [11]:
df['label'] = df.subreddit.replace(label_dict)
df.head()

,subreddit,text,label
0,wallstreetbets,Any good traded for a fellow ape,0
1,wallstreetbets,The top is in boys.,0
2,wallstreetbets,HMHC May 22.5C 💎🤲💎,0
3,wallstreetbets,This morning I bought $8K of $40 TWTR puts expiring end of month. Down 30% so far. let's see what happens.,0
4,wallstreetbets,Loading… Fresh New Sanctions,0


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.20, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [13]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['subreddit', 'label', 'data_type']).count()

text
subreddit        label data_type      
GenZ             3     train       400
                       val         100
copypasta        2     train       400
                       val         100
teenagers        1     train       400
                       val         100
unpopularopinion 4     train       400
                       val         100
wallstreetbets   0     train       400
                       val         100

## Load Slang Data
Data are a direct translation of slang to their non-slang synonym

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
slang = pd.read_csv('/content/gdrive/MyDrive/w266/final_project/slangit_group_edit - slangit_kevin_edit.csv')
slang.head(20)

,Slang Term,Meaning
0,@@-o,tattletale
1,@teotd,at the end of the day
2,^5,high five
3,0773h,hello
4,10m,10 minutes
5,10q,thank you
6,10x,10 times
7,1337,leet speak
8,143,i love you
9,1up,extra life


In [16]:
slangit_dict = slang.set_index('Slang Term').to_dict()
slangit_dict = slangit_dict['Meaning']

In [17]:
keys_values = slangit_dict.items()
slangit_dict = {str(key): str(value) for key, value in keys_values}

In [18]:
def slang_lookup(text, dictionary):
    
    #try to make any case
    #try to make it only words with spaces
    text = text.lower()
    pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in slangit_dict.keys()) + r')(?!\w)')
    result = pattern.sub(lambda x: dictionary[x.group()], text)

    return result

In [19]:
my_text = 'I watched the UNC game at a bar b/c YOLO, FTW'

print(slang_lookup(my_text, slangit_dict))

i watched the unc game at a bar because you only live once, for the win


In [20]:
def slang_lookup(text, dictionary):
    
    #try to make any case
    #try to make it only words with spaces
    pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in slangit_dict.keys()) + r')(?!\w)')
    result = pattern.sub(lambda x: dictionary[x.group()], text)

    return result

In [21]:
df_train = df[df.index.isin(X_train)]
df_train['text_deslanged'] = df_train['text'].apply(lambda x: slang_lookup(x, slangit_dict))
df_train['text_deslanged'].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0                                                                                                                         Any good traded for a fellow ape
2                                                                                                                                       HMHC May 22.5C 💎🤲💎
3    This morning I bought $8K onlyfans $40 TWTR puts expiring end onlyfans month. Down 30% significant other far. leaving early today's see what happens.
4                                                                                                                             Loading… Fresh New Sanctions
5                                                     Last Year When I Yolo’defense on AMC with some Margin…allow natural death I’later loser do it again.
Name: text_deslanged, dtype: object

In [22]:
#do same thing to val
df_val = df[df.index.isin(X_val)]
df_val['text_deslanged'] = df_val['text'].apply(lambda x: slang_lookup(x, slangit_dict))
df_val['text_deslanged'].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    The top lane is in 

In [23]:
df_val.head()

,subreddit,text,label,data_type,text_deslanged
1,wallstreetbets,The top is in boys.,0,val,The top lane is in boys.
13,wallstreetbets,"Posters can record the trades they plan to make the evening prior to active next day, with what kind of trade, the parameters, equity etc\n\nAfter the day’s trading is done, then an update is done to capture the outcomes per redditor. Over time, we capture who is ‘the best’, ranked by an agreed metric and this becomes a jump off point to dig into what truly talented traders do differently. \n\nThere can be the option to just select buy vs sell per position, for those who don’t want to share money info.\n\nThoughts..? I think it would be awesome",0,val,"Posters can record the trades they plan to make the evening prior to active next day, with what kind onlyfans destroy or counter an opponent's card with your own, the parameters, equity etc\n\nAfter the day’s trading is i can't deal with it, then an update is i can't deal with it to capture the outcomes per a reddit user. Over time, whatever capture world health organization is ‘the best’, ranked by an agreed metric allow natural death this becomes a jump off point to like into what truly talented traders do differently. \n\nThere can be the option to just select buy versus sell per position, for those world health organization don’t want to share money information.\n\nThoughts..? I think it would be awesome"
20,wallstreetbets,"The DJIA index, since its peak of November 2021, tilted with massive purchases appearing that propelled it to its all-time high in December. Every time you lean in, massive purchases appear. Also in February and March 2022. The war scenario is sending international capital into the U.S. stock market. \n\nAre we in a phase of accumulation and consolidation for new all-time highs? And also a historic vertical bullish rally? \n\n&amp;#x200B;\n\nhttps://preview.redd.it/qh29qh29zkr81.png?width=1299&amp;format=png&amp;auto=webp&amp;s=ba5e98c3d3f5f719275ed049291cbdf27d82386f",0,val,"The DJIA index, since its peak onlyfans November 2021, aggravated with massive purchases appearing that propelled it to its all-time high in December. Every time you lean in, massive purchases appear. Also in February allow natural death March 2022. The wins above replacement scenario is sending international capital into the U.S. stock market. \n\nAre whatever in a phase onlyfans accumulation allow natural death consolidation for new all-time highs? And also a historic vertical positive outlook rally? \n\n&to be loud and angry;#x200B;\n\nhypertext transfer protocol secure://preview.redd.it/qh29qh29zkr81.png?width=1299&to be loud and angry;format=png&to be loud and angry;auto=webp&to be loud and angry;s=ba5e98c3d3f5f719275ed049291cbdf27d82386f"
21,wallstreetbets,"Hi there, I’m a studying international business management at university, I am doing my dissertation on trading in the stock market. I am investigating into the causes of the GameStop short squeeze and how social media impacted this. Any responses to the survey would be greatly appreciated. :) :)[Survey](https://docs.google.com/forms/d/e/1FAIpQLScL9QeHPoqvOlpM0XEP2W7AlNerOUUk-B2i637S0flq6PXu_g/viewform)",0,val,"Hi there, I’m a studying international business management at university, I antemeridian doing my dissertation on trading in the stock market. I antemeridian investigating into the causes onlyfans the GameStop short squeeze allow natural death how social media impacted this. Any responses to the survey would be greatly appreciated. :) :)[Survey](hypertext transfer protocol secure://docs.to search the internet.com/forms/defense/e/1FAIpQLScL9QeHPoqvOlpM0XEP2W7AlNerOUUk-B2i637S0flq6PXu_g/viewform)"
34,wallstreetbets,"Holding about 800 shares. Forecasting the rest of the year and seeing something I don’t like, but maybe one of you is better informed?\n\nNVDA is selling between 25% and 57% of all graphics cards to cryptominers. In August moves away from proof of work, meaning no more mining 

In [24]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


train_data = df_train[['text_deslanged', 'subreddit']]
train_data['prefix'] = 'multilabel classification'
train_df = train_data[['prefix', 'text_deslanged', 'subreddit']]
train_df = train_df.rename(columns={'prefix': 'prefix', 'text_deslanged': 'input_text', 'subreddit': 'target_text'})

eval_data = df_val[['text_deslanged', 'subreddit']]
eval_data['prefix'] = 'multilabel classification'
eval_df = eval_data[['prefix', 'text_deslanged', 'subreddit']]
eval_df = eval_df.rename(columns={'prefix': 'prefix', 'text_deslanged': 'input_text', 'subreddit': 'target_text'})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":


In [25]:
eval_df.head()

,prefix,input_text,target_text
1,multilabel classification,The top lane is in boys.,wallstreetbets
13,multilabel classification,"Posters can record the trades they plan to make the evening prior to active next day, with what kind onlyfans destroy or counter an opponent's card with your own, the parameters, equity etc\n\nAfter the day’s trading is i can't deal with it, then an update is i can't deal with it to capture the outcomes per a reddit user. Over time, whatever capture world health organization is ‘the best’, ranked by an agreed metric allow natural death this becomes a jump off point to like into what truly talented traders do differently. \n\nThere can be the option to just select buy versus sell per position, for those world health organization don’t want to share money information.\n\nThoughts..? I think it would be awesome",wallstreetbets
20,multilabel classification,"The DJIA index, since its peak onlyfans November 2021, aggravated with massive purchases appearing that propelled it to its all-time high in December. Every time you lean in, massive purchases appear. Also in February allow natural death March 2022. The wins above replacement scenario is sending international capital into the U.S. stock market. \n\nAre whatever in a phase onlyfans accumulation allow natural death consolidation for new all-time highs? And also a historic vertical positive outlook rally? \n\n&to be loud and angry;#x200B;\n\nhypertext transfer protocol secure://preview.redd.it/qh29qh29zkr81.png?width=1299&to be loud and angry;format=png&to be loud and angry;auto=webp&to be loud and angry;s=ba5e98c3d3f5f719275ed049291cbdf27d82386f",wallstreetbets
21,multilabel classification,"Hi there, I’m a studying international business management at university, I antemeridian doing my dissertation on trading in the stock market. I antemeridian investigating into the causes onlyfans the GameStop short squeeze allow natural death how social media impacted this. Any responses to the survey would be greatly appreciated. :) :)[Survey](hypertext transfer protocol secure://docs.to search the internet.com/forms/defense/e/1FAIpQLScL9QeHPoqvOlpM0XEP2W7AlNerOUUk-B2i637S0flq6PXu_g/viewform)",wallstreetbets
34,multilabel classification,"Holding about 800 shares. Forecasting the rest onlyfans the year allow natural death seeing something I don’t like, but maybe one onlyfans you is better informed?\n\nNVDA is selling between 25% allow natural death 57% onlyfans all graphics cards to cryptominers. In August moves away from proof onlyfans a scripted event, meaning know more mining it. This is to get rid onlyfans the energy usage to are you in? the currency. I’very emotional been watching for 3 months allow natural death the amount onlyfans cards showing up on eBay is growing drastically allow natural death getting cheaper. Miners are trying to cash out their cards before they take a big hit in value. \n\nNVDA has enjoyed stock sellouts because onlyfans chip shortages allow natural death I feel mainly because onlyfans miners. The cost onlyfans the highest end GPU’s has almost tripled because onlyfans this demand. NVDA has been trying to to permanently gain additional mana production to capitalize allow natural ...",wallstreetbets


In [26]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [27]:
model_args = T5Args()
model_args.num_train_epochs = 10
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True
torch.cuda.memory_summary(device=None, abbreviated=False)
model_args.per_gpu_train_batch_size = 128

model = T5Model("t5", "t5-base", args=model_args, use_cuda=True)


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

In [28]:
def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])

In [29]:
model.train_model(train_df, eval_data=eval_df, matches=count_matches)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-base_cached_1282000
INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/250 [00:00<?, ?it/s]

...

Generating outputs:   0%|          | 0/63 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/500 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': 0.5425466282393909, 'matches': 350}
INFO:simpletransformers.t5.t5_model: Training of t5-base model complete. Saved to outputs/.


['wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstre

(2500,
 {'eval_loss': [0.19366755490265194,
   0.22722635294739452,
   0.26694851292903343,
   0.2698771390539136,
   0.334214009167183,
   0.3502710684302396,
   0.4636661711082857,
   0.41178933775702875,
   0.41178933775702875,
   0.4652282296169904,
   0.5425466282393909],
  'global_step': [250,
   500,
   750,
   1000,
   1250,
   1500,
   1750,
   2000,
   2000,
   2250,
   2500],
  'matches': [356, 360, 326, 355, 376, 382, 362, 372, 372, 363, 350],
  'train_loss': [0.2865143418312073,
   0.3758179545402527,
   0.14878831803798676,
   0.031661342829465866,
   0.0005758012994192541,
   0.10422786325216293,
   0.04754133149981499,
   0.00029718081350438297,
   0.00029718081350438297,
   0.043506670743227005,
   0.007107068784534931]})

In [30]:
my_t5 = model.eval_model(eval_df, matches=count_matches)
print(my_t5)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-base_cached_128500


Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/63 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/500 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': 0.5425466282393909, 'matches': 350}


['wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstreetbets', 'wallstre

In [31]:
#get predictions included in df
preds = model.predict(list(eval_df['input_text']))
eval_df['t5_prediction'] = preds

Generating outputs:   0%|          | 0/63 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/500 [00:00<?, ?it/s]

In [32]:
eval_df['t5_prediction'] = preds
eval_df.head()

,prefix,input_text,target_text,t5_prediction
1,multilabel classification,The top lane is in boys.,wallstreetbets,GenZ
13,multilabel classification,"Posters can record the trades they plan to make the evening prior to active next day, with what kind onlyfans destroy or counter an opponent's card with your own, the parameters, equity etc\n\nAfter the day’s trading is i can't deal with it, then an update is i can't deal with it to capture the outcomes per a reddit user. Over time, whatever capture world health organization is ‘the best’, ranked by an agreed metric allow natural death this becomes a jump off point to like into what truly talented traders do differently. \n\nThere can be the option to just select buy versus sell per position, for those world health organization don’t want to share money information.\n\nThoughts..? I think it would be awesome",wallstreetbets,wallstreetbets
20,multilabel classification,"The DJIA index, since its peak onlyfans November 2021, aggravated with massive purchases appearing that propelled it to its all-time high in December. Every time you lean in, massive purchases appear. Also in February allow natural death March 2022. The wins above replacement scenario is sending international capital into the U.S. stock market. \n\nAre whatever in a phase onlyfans accumulation allow natural death consolidation for new all-time highs? And also a historic vertical positive outlook rally? \n\n&to be loud and angry;#x200B;\n\nhypertext transfer protocol secure://preview.redd.it/qh29qh29zkr81.png?width=1299&to be loud and angry;format=png&to be loud and angry;auto=webp&to be loud and angry;s=ba5e98c3d3f5f719275ed049291cbdf27d82386f",wallstreetbets,wallstreetbets
21,multilabel classification,"Hi there, I’m a studying international business management at university, I antemeridian doing my dissertation on trading in the stock market. I antemeridian investigating into the causes onlyfans the GameStop short squeeze allow natural death how social media impacted this. Any responses to the survey would be greatly appreciated. :) :)[Survey](hypertext transfer protocol secure://docs.to search the internet.com/forms/defense/e/1FAIpQLScL9QeHPoqvOlpM0XEP2W7AlNerOUUk-B2i637S0flq6PXu_g/viewform)",wallstreetbets,wallstreetbets
34,multilabel classification,"Holding about 800 shares. Forecasting the rest onlyfans the year allow natural death seeing something I don’t like, but maybe one onlyfans you is better informed?\n\nNVDA is selling between 25% allow natural death 57% onlyfans all graphics cards to cryptominers. In August moves away from proof onlyfans a scripted event, meaning know more mining it. This is to get rid onlyfans the energy usage to are you in? the currency. I’very emotional been watching for 3 months allow natural death the amount onlyfans cards showing up on eBay is growing drastically allow natural death getting cheaper. Miners are trying to cash out their cards before they take a big hit in value. \n\nNVDA has enjoyed stock sellouts because onlyfans chip shortages allow natural death I feel mainly because onlyfans miners. The cost onlyfans the highest end GPU’s has almost tripled because onlyfans this demand. NVDA has been trying to to permanently gain additional mana production to capitalize allow natural ...",wallstreetbets,unpopularopinion
